In [2]:
from datascience import *

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')


import os
import networkx as nx
from networkx.algorithms import bipartite
import pandas as pd

## Affiliation and bipartite graphs

This dataset comes from [Bonica (2016)](https://www.cambridge.org/core/journals/business-and-politics/article/avenues-of-influence-on-the-political-expenditures-of-corporations-and-their-directors-and-executives/F9EF7632D19B480BACEA337FF0516ADE)

In [30]:
data_dir = "~/Dropbox/data-explore/dime-political-contributions/ceos/dataverse_files"

In [31]:
dfa = pd.read_csv(os.path.join(data_dir, "bod_fortune_500_DIME_cont_records.csv"))
dfa.head()

,dime.cid,ticker,corpname,cfscore,cycle,transaction.id,transaction.type,amount,date,contributor.name,...,contributor.employer,contributor.address,contributor.city,contributor.state,contributor.zipcode,latitude,longitude,dime.rid,recipient.name,recipient.party
0,2.626837e+10,AA,Alcoa Inc,NaN,2012,e:ind:2012:75303306,15,5000.00,10/31/11,"kleinfeld, klaus",...,alcoa inc,3 rye rd,port chester,NY,105735313,40.986977,-73.663765,comm155160,ALCOA INC EMPLOYEES' VOLUNTARY POLITICAL ACTIO...,NaN
1,2.626837e+10,AA,Alcoa Inc,NaN,2012,e:ind:2012:110036574,15,416.66,1/31/12,"kleinfeld, klaus",...,alcoa inc,3 rye rd,port chester,NY,105735313,40.986977,-73.663765,comm155160,ALCOA INC EMPLOYEES' VOLUNTARY POLITICAL ACTIO...,NaN
2,2.626837e+10,AA,Alcoa Inc,NaN,2012,e:ind:2012:120223355,15,416.66,2/29/12,"kleinfeld, klaus",...,alcoa inc,3 rye rd,port chester,NY,105735313,40.986977,-73.663765,comm155160,ALCOA INC EMPLOYEES' VOLUNTARY POLITICAL ACTIO...,NaN
3,2.626837e+10,AA,Alcoa Inc,NaN,2012,e:ind:2012:12041701950641,15,416.66,3/30/12,"kleinfeld, klaus",...,alcoa inc,3 rye rd,port chester,NY,105735313,40.986977,-73.663765,comm155160,ALCOA INC EMPLOYEES' VOLUNTARY POLITICAL ACTIO...,NaN
4,2.626837e+10,AA,Alcoa Inc,NaN,2012,e:ind:2012:12052105804198,15,416.66,4/30/12,"kleinfeld, klaus",...,alcoa inc,3 rye rd,port chester,NY,105735313,40.986977,-73.663765,comm155160,ALCOA INC EMPLOYEES' VOLUNTARY POLITICAL ACTIO...,NaN


In [32]:
dfb = pd.read_csv(os.path.join(data_dir, "bod_fortune_500_DIME.csv"))
dfb.head()

,dime.cid,corp.person.id,ticker,corp.name,last.name,first.name,middle.name,age,gender,ceo,...,total.2004,total.2006,total.2008,total.2010,total.2012,to.incumbs,to.open.seat,to.challs,to.winner,to.losers
0,3950990771,548428,A,Agilent Technologies,clark,paul,n,65,M,0,...,8641.0,0.0,15000.0,0.0,0.0,0.0,9600.0,0.0,0.0,0.0
1,3247157790,652783,A,Agilent Technologies,yamada,tadataka,NaN,66,M,0,...,3000.0,4000.0,0.0,0.0,0.0,6000.0,0.0,0.0,4000.0,2000.0
2,3027018565,598067,A,Agilent Technologies,cullen,james,g,69,M,0,...,0.0,0.0,0.0,0.0,0.0,26300.0,8300.0,8250.0,17300.0,7550.0
3,4598973803,612576,A,Agilent Technologies,lawrence,david,mckinnon,73,M,0,...,4250.0,805.0,3350.0,8050.0,3750.0,9900.0,4150.0,3000.0,7400.0,1500.0
4,5341881827,181606,A,Agilent Technologies,herbold,robert,j,69,M,0,...,11825.0,0.0,0.0,2900.0,30200.0,17600.0,2000.0,11650.0,11700.0,11300.0


In [48]:
pd.pivot_table(dfa, index=['recipient.party'], aggfunc='count')

,amount,cfscore,contributor.address,contributor.city,contributor.employer,contributor.fname,contributor.lname,contributor.mname,contributor.name,contributor.occupation,...,cycle,date,dime.cid,dime.rid,latitude,longitude,recipient.name,ticker,transaction.id,transaction.type
recipient.party,,,,,,,,,,,,,,,,,,,,,
,534,527,58,534,419,533,533,190,534,449,...,534,534,534,534,527,527,534,531,534,534
0,132,132,128,132,132,132,132,120,132,132,...,132,132,132,132,132,132,132,132,132,70
100,81026,79429,48679,80776,65979,81001,81022,39843,81026,49245,...,81026,80443,81026,80970,80916,80916,81026,80572,81026,58941
200,104256,102221,63419,103947,81584,104213,104251,52476,104256,61442,...,104256,103739,104256,104210,104070,104070,104256,103738,104256,76114
328,691,682,552,691,541,691,691,293,691,481,...,691,690,691,691,691,691,691,690,691,376
402,11,11,11,11,9,11,11,1,11,9,...,11,11,11,11,11,11,11,11,11,2
C0001,8,8,8,8,8,8,8,8,8,8,...,8,8,8,0,8,8,0,8,8,0
CIT,18,18,18,18,18,18,18,10,18,18,...,18,18,18,18,18,18,18,18,18,18
CON,2,2,2,2,2,2,2,1,2,2,...,2,2,2,2,2,2,2,2,2,2


In [43]:
dfa.shape

(313914, 26)

In [45]:
534+132+81026+104256+691+11+8+18+2+2+1+2750+5+1635+2660+1+690+5+31+13657

208115

In [10]:
dfa.shape

(313914, 26)